In [1]:
import pymongo as pym
import pandas as pd
import requests
import nltk
from bs4 import BeautifulSoup
#from bs4 import BeautifulSoup as bs
import urllib.request, urllib.parse, urllib.error
from newspaper import Article 

In [4]:
url=["https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml","https://blockchain.news/RSS/","https://news.bitcoin.com/feed/"]

In [5]:
for u in url:
        #Making Connection with mongodb
        client = pym.MongoClient('mongodb://localhost:27017/')
        #creating db
        db = client["Rss_Feed"]
        #creating tables
        rss_feeds_table = db["Rss_Feed_Info"]


        resp=requests.get(u)
        #soup = bs(resp.text, 'html.parser')
        soup=BeautifulSoup(resp.content,features='xml')
        news = []
        items=soup.findAll('item')
        #nltk.download('punkt')
        item = iter(items)
        for item in items:
            news_item={}

            #Check if there is document with PubDate and Title specified
            if db.Rss_Feed_Info.find_one({"PubDate":item.pubDate.text,"Title":item.title.text}):
                #print("Data present")
                #If The document with specified PubDate and Title is present the news_item store null values
                news_item['Title']=None
                news_item['Link']=None
                news_item['Description']=None
                news_item['PubDate']=None
                #news_item['Category']=item.category.text
                news_item['Keyword']=None
                #next(item)
            else:  
                #If The document with specified PubDate and Title is not present news_item stores the values
                news_item['Title']=item.title.text
                news_item['Link']=item.link.text
                news_item['Description']=item.description.text
                news_item['PubDate']=item.pubDate.text
                #news_item['Category']=item.category.text
                a = item.link.text
                article = Article(a)
                article.download()
                article.parse() 
                article.nlp()
                news_item['Keyword']=article.keywords
            news.append(news_item)

        df = pd.DataFrame(news)
        #df.head(26)
       
        # Create Dict() to send to MongoDB
        rss_feed_dict = df.to_dict('record')
        rss_feeds_table.insert_many(rss_feed_dict)

        #Removing Null Values
        db.Rss_Feed_Info.delete_many({ "Title": None , "Link" : None , "Description": None, "PubDate": None})
        #db.Rss_Feed_Info_keyword.find({},{"PubDate":-1}).sort("PubDate",-1)

ArticleException: Article `download()` failed with 404 Client Error: Not Found for url: https://blockchain.news/404?q=ethereum?s-median-fees-drop-by-88.7-hitting-a-two-month-low on URL https://Blockchain.News/news/ethereum’s-median-fees-drop-by-88.7-hitting-a-two-month-low